In [1]:
from modules.data import Data
from modules.rag import resolve_context
from modules.utils import resumable, save_progress

from tqdm import tqdm

In [2]:
data = Data()
data_test = data.get_split("test")

2024-07-16 17:26:50.133 | INFO     | modules.data:load_data:18 - load dev data, len: 1000
2024-07-16 17:26:50.140 | INFO     | modules.data:load_data:18 - load test data, len: 1000
2024-07-16 17:26:50.176 | INFO     | modules.data:load_data:18 - load train data, len: 5000


In [3]:
context_template = """
产品名：{product_name}
条款：{clause}
"""

In [4]:
@resumable(progress_file="./cache/progress_test.json")
def process_data_with_rag(data, context_template, start_idx=0, progress_file=None):
    for idx, item in tqdm(
        enumerate(data[start_idx:], start=start_idx), initial=start_idx, total=len(data)
    ):
        product_name, raw_context, query = item["产品名"], item["条款"], item["问题"]
        if len(raw_context) > 500:
            retrieved_nodes = resolve_context(
                idx=idx, raw_context=raw_context, query=query
            )
            context = ""
            for node_with_score in retrieved_nodes:
                context += node_with_score.node.text
        else:
            context = raw_context
        data[idx]["context"] = context_template.format(
            product_name=product_name, clause=context
        )

        # Save progress after each item is processed
        if progress_file:
            save_progress(progress_file, data, ensure_ascii=False, indent=4)

    return data


# 使用装饰器的函数
data_test_new = process_data_with_rag(data=data_test, context_template=context_template)

Resuming from index 3


  0%|          | 3/1000 [00:00<?, ?it/s]2024-07-16 17:26:50.210 | INFO     | modules.rag:resolve_context:15 - idx: 3, query: 如果在上述情形或期间内发生保险事故，保险公司如何处理已交保费？, raw_context: 责任免除 因下列一个或者多个情形引起的保险事故，本公司不承担相应的保险责任：1)投保人对被保险人的故意杀害、故意伤害；2)被保险人故意犯罪或者抗拒依法采取的刑事强制措施；3）被保险人故意自伤、或自本合同成立或者本合同效力恢复之日起2年内自杀，但被保险人自杀时为无?事行为能力人的除外；4）被保险人患有遗传性疾病（?7.7），先天性畸形、变形或染色体异常（?7.8）（在本合同有效期内罹患本条款第7.18条“轻度疾病”释义中约定的第20项轻度疾病、第7.20条“重度疾病”释义中约定的第35、40、45、56、61、70、80、82、99、100、101项重度疾病、第7.21条“特定疾病”释义中约定的第10项特定疾病除外）；5）战争、军事冲突、暴乱或武装叛乱；6)核爆炸、核辐射或核污染。被保险人在下列期间内发生的保险事故，本公司不承担相应的保险责任：1)被保险人服用、吸?或注射毒品（?7.9）；2)被保险人酒后驾驶(?7.10）、无合法有效驾驶证驾驶（?7.11），或驾驶无合法有效行驶证（?7.12）的机动?（?7.13）；3)被保险人感染艾滋病病毒或患艾滋病（?7.14）（在本合同有效期内罹患本条款第7.20条“重度疾病”释义中约定的第31、81项重度疾病除外）。被保险人因上述情形或在上述期间内发生本合同约定的保险事故的，本合同效力终止，本公司向投保人退还本合同的现金价值（?7.15）。但因投保人对被保险人的故意杀害、故意伤害导致被保险人发生本合同约定的保险事故的，本公司向其他权利人退还本合同的现金价值，其他权利人按照被保险人（第一顺位）、被保险人继承人（第二顺位）的顺序确定。其他免责条款除本条款第2.4条“责任免除”外，本合同中还有一些免除本公司责任的条款，详?本条款第2.3条“保险责任”、第3.3条“犹豫期后解除合同的手续及?险”、第4.4条“合同效力的中止和恢复”、第6.7条“款项扣除”和第7条“名词释义”等部分中以黑体字加下划线

2024-07-16 17:26:50.670 | INFO     | modules.rag:resolve_context:36 - #nodes: 4


Generating embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

  1%|          | 12/1000 [00:52<25:20,  1.54s/it]  2024-07-16 17:27:42.594 | INFO     | modules.rag:resolve_context:15 - idx: 12, query: 如果被保险人从事高风险运动而发生意外，保险公司是否承担赔偿责任？, raw_context: 救援类通用责任免除 因下列情形之一造成的运送或运返费用、直接或间接造成被保险人身故或住院的、直接或间接造成被保险人身故或发生的费用，保险人不承担给付保险金责任：（1）被保险人接受包括美容、整容、整形手术在内的任何医疗行为而造成的意外；（2）被保险人受酒精、毒品、管制药物的影响，但遵医嘱使用药物的情形不在此限；（3）任何生物、化学、原子能武器，原子能或核能装置所造成的爆炸、灼伤、污染或辐射；（4）被保险人故意犯罪或抗拒依法采取的刑事强制措施；（5）被保险人存在精神和行为障碍（以世界卫生组织颁布的《疾病和有关健康问题的国际统计分类（ICD-10）》为准）；（6）被保险人从事高风险运动；（7）被保险人酒后驾驶、无有效驾驶证驾驶或驾驶无有效行驶证的机动交通工具；（8）被保险人感染艾滋病病毒或患艾滋病。
因下列情形之一造成的运送或运返费用、直接或间接造成被保险人身故或住院的、直接或间接造成被保险人身故或发生的费用，保险人不承担给付保险金责任： （1）被保险人接受包括美容、整容、整形手术在内的任何医疗行为而造成的意外； （2）被保险人受酒精、毒品、管制药物的影响，但遵医嘱使用药物的情形不在此限； （3）任何生物、化学、原子能武器，原子能或核能装置所造成的爆炸、灼伤、污染或辐射； （4）被保险人故意犯罪或抗拒依法采取的刑事强制措施； （5）被保险人存在精神和行为障碍（以世界卫生组织颁布的《疾病和有关健康问题的国际统计分类（ICD-10）》为准）； （6）被保险人从事高?险运动； （7）被保险人酒后驾驶、无有效驾驶证驾驶或驾驶无有效行驶证的机动交通工具； （8）被保险人感染艾滋病病毒或患艾滋病。
2024-07-16 17:27:42.600 | INFO     | modules.rag:resolve_context:36 - #nodes: 4


...

 27%|██▋       | 269/1000 [16:05<17:43,  1.46s/it]  2024-07-16 17:42:55.944 | INFO     | modules.rag:resolve_context:15 - idx: 269, query: 被保险人的哪些既往症状会导致保险人不负责给付保险金？, raw_context: 责任免除 本保险合同的责任免除如下：一、主险合同无效或失效，保险人不负任何给付保险金责任。二、主险合同中列明的“责任免除”事项，也适用于本保险合同。三、在下列情形下，保险人不负任何给付保险金责任：（一）被保险人非因主险合同保险责任范围内的意外伤害事故而发生的救护车费用；（二）被保险人送往非二级以上或保险人认可的医疗机构而发生的救护车费用，但意外伤害急救至就近医院不在此限。
本保险合同的责任免除如下：一、主险合同无效或失效，保险人不负任何给付保险金责任。二、主险合同中列明的“责任免除”事项，也适用于本保险合同。三、下列情形下，保险人不负任何给付保险金责任：    （一）被保险人投保前已有的既往症；    （二）被保险人先天性关节脱位、病理性脱位、习惯性脱位、陈旧性脱位或复发性脱位；   （三）被保险人病理性骨折（指有病骨骼遭受轻微外力即发生的骨折）或被诊断为骨质疏松并因该病症而导致的骨折或关节脱位。
下列任一情形下，保险人对被保险人身故、残疾不负任何给付保险金责任：（一）被保险人精神失常或精神错乱期间；（二）战争、军事行动、暴动或武装叛乱期间；（三）被保险人醉酒或受毒品、管制药物的影响期间；（四）被保险人酒后驾车、无有效驾驶证驾驶或驾驶无有效行驶证的机动车期间。
因下列原因之一，直接或间接造成被保险人身故、残疾的，保险人不负任何给付保险金责任：（一）投保人对被保险人的故意杀害或伤害；（二）被保险人自致伤害或自杀，但被保险人自杀时为无民事行为能力人的除外；（三）因被保险人挑衅或故意行为而导致的打斗、被袭击或被谋杀；（四）被保险人违法、犯罪或者抗拒依法采取的刑事强制措施；（五）被保险人因疾病导致的伤害，包括但不限于猝死、食物中毒、高原反应、中暑、病毒和细菌感染（意外伤害导致的伤口感染不在此限）；（六）被保险人因妊娠、流产、分娩导致的伤害，但意外伤害所致的流产或分娩不在此限；（七）被保险人因药物过敏、整容手术、内外科手术

Generating embeddings:   0%|          | 0/14 [00:00<?, ?it/s]

 27%|██▋       | 271/1000 [16:21<50:12,  4.13s/it]  2024-07-16 17:43:11.841 | INFO     | modules.rag:resolve_context:15 - idx: 271, query: 保险责任开始前，如果投保人要求解除保险合同，投保人需要支付多少退保手续费？, raw_context: 其他事项 保险责任开始前，投保人要求解除保险合同的，应当向保险人支付相当于保险费 5%的退保手续费，保险人应当退还剩余部分保险费；保险人要求解除保险合同的，不得向投保人收取手续费并应退还已收取的保险费。保险责任开始后，投保人要求解除保险合同的，自通知保险人之日起，保险合同解除，保险人将退回未满期保费，计算公式：退回未满期保费=保险费*（1-保险单已经过天数/保险期间天数）。
保险责任开始前，投保人要求解除保险合同的，应当向保险人支付相当于保险费5%的退保手续费，保险人应当退还剩余部分保险费；保险人要求解除保险合同的，不得向投保人收取手续费并应退还已收取的保险费。保险责任开始后，投保人要求解除保险合同的，自通知保险人之日起，保险合同解除，保险人将退回未满期保费，计算公式：未满期保费=保险费 *（1-保险单已经过天数/保险期间天数）。
保险责任开始前，投保人要求解除保险合同的，应当向保险人支付相当于保险费5%的退保手续费，保险人应当退还剩余部分保险费；保险人要求解除保险合同的，不得向投保人收取手续费并应退还已收取的保险费。保险责任开始后，投保人要求解除保险合同的，自通知保险人之日起，保险合同解除，保险人按短期费率计收保险责任开始之日起至合同解除之日止期间的保险费，并退还剩余部分保险费；保险人也可提前十五日向投保人发出解约通知书解除本保险合同，保险人按照保险责任开始之日起至合同解除之日止期间与保险期间的日比例计收保险费，并退还剩余部分保险费。
2024-07-16 17:43:11.851 | INFO     | modules.rag:resolve_context:36 - #nodes: 4


Generating embeddings:   0%|          | 0/4 [00:00<?, ?it/s]

 27%|██▋       | 272/1000 [16:46<2:07:21, 10.50s/it]2024-07-16 17:43:37.187 | INFO     | modules.rag:resolve_context:15 - idx: 272, query: 保险人为了核验客户的投保资格，可能会向哪些第三方主体申请核验？, raw_context: 基于您购买保险产品业务所需，保险人将收集以下信息 （1）您提供给保险人供实现保险服务的信息（如投保时填写的个人信息，包括姓名、性别、联系方式、出生日期、证件类型、证件号码、证件有效期、国籍、身高体重、电子邮箱、长期居住地、详细地址、学历、工作单位信息、职业及收入信息、婚姻状况）；（2）您享受保险服务（如核保核赔、客户回访、业务保全、保险理赔等）所产生或需要的个人身份信息、健康医疗信息、保单信息等；（3）您依法留存在保险人的合作方、保险人的关联公司及其合作方、行政机关及其授权单位、司法机关、公安部门、司法鉴定中心、银保监会及其下设机构、人社相关机构、医疗机构、保险公司、工作单位等等组织机构处的保险产品和服务相关的必要信息（如体检报告、诊断报告、病历等健康医疗信息等）；（4）在提供投保等保险服务过程中，保险人可能会向第三方主体申请核验您的身份信息的真实性，用于核验您是否具备投保资格；（5）为防范保险欺诈等行为，保险人还可能向行政机关、司法机关、征信机构、医疗机构以及其他合法了解掌握相关信息的自然人和法人，收集或查询您的身份、财产、社保、健康状况、保险服务等相关信息，以核验是否存在保险欺诈等行为。请您知悉，依据《中华人民共和国个人信息保护法》，保险人处理以下个人信息无需征得您的同意：（1）为订立、履行个人作为一方当事人的合同所必需，或者按照依法制定的劳动规章制度和依法签订的集体合同实施人力资源管理所必需；（2）为履行法定职责或者法定义务所必需；（3）为应对突发公共卫生事件，或者紧急情况下为保护自然人的生命健康和财产安全所必需；（4）为公共利益实施新闻报道、舆论监督等行为，在合理的范围内处理个人信息；（5）依照本法规定在合理的范围内处理个人自行公开或者其他已经合法公开的个人信息；（6）法律、行政法规规定的其他情形。
2024-07-16 17:43:37.196 | INFO     | modules.rag